In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
text=open(path_to_file,'rb').read().decode(encoding='utf-8')
print('Length of text:{} characters'.format(len(text)))

Length of text:1115394 characters


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



Encoding

In [6]:
vocab=sorted(set(text))
char2idx={u:i for i,u in enumerate(vocab)}
idx2char=np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int=text_to_int(text)

In [7]:
print("Text:",text[:13])
print("Encoded:",text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [8]:
def int_to_text(ints):
  try:
    ints=ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


Creating Traning examples

In [10]:
seq_length=100
examples_per_epoch=len(text)//(seq_length+1)
char_dataset=tf.data.Dataset.from_tensor_slices(text_as_int)

In [11]:
sequences=char_dataset.batch(seq_length+1,drop_remainder=True)

In [12]:
def split_input_target(chunk):
  input_text=chunk[:-1]
  target_text=chunk[1:]
  return input_text,target_text
dataset=sequences.map(split_input_target)

In [13]:
for x,y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [16]:
BATCH_SIZE=64
VOCAB_SIZE=len(vocab)
EMBEDDING_DIM=256
RNN_UNITS=1024
BUFFER_SIZE=10000
data=dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True)

Building the model

In [22]:
def build_model(vocab_size,embedding_dim,rnn_units,batch_size):
  model=tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size,embedding_dim),
      tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model
model=build_model(VOCAB_SIZE,EMBEDDING_DIM,RNN_UNITS,BATCH_SIZE)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Creating a loss function

In [23]:
for input_example_batch,target_example_batch in data.take(1):
  example_batch_predictions=model(input_example_batch)
  print(example_batch_predictions.shape,"# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [24]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-2.55050836e-03 -2.92298105e-03  5.10155829e-03 ...  7.62772879e-06
    4.72863391e-03  1.73572058e-04]
  [-1.86973577e-03 -3.54210194e-03  6.36438141e-03 ... -1.27401645e-03
   -3.56358301e-04 -4.75702807e-03]
  [-6.94843289e-03 -1.46182114e-03  5.85578894e-03 ... -4.10201540e-03
    1.53742172e-03  1.43814177e-04]
  ...
  [ 6.28186343e-03  2.70135421e-03  3.05247202e-04 ... -6.83615636e-03
   -1.29747316e-02 -5.85907651e-03]
  [ 8.79418291e-03  1.09492068e-03  2.30789720e-03 ... -7.92460691e-04
   -1.32958647e-02 -4.76150727e-03]
  [ 9.02094413e-03  5.92649973e-04  9.38136131e-04 ... -1.31660956e-03
   -7.19960220e-03 -2.22864398e-03]]

 [[ 3.27006495e-03 -1.32703326e-06 -3.63947335e-03 ...  4.87749407e-04
    4.77255788e-03  1.33166718e-03]
  [ 6.19166438e-03  4.21301927e-03 -4.80713649e-03 ... -1.31418731e-03
    1.55108783e-03  2.45953701e-03]
  [ 7.60754431e-03  4.19441704e-03 -5.05373999e-03 ... -4.05292166e-03
   -1.29109656e-03  1.26812549e-03]
  ...
  [-4.267

In [25]:
pred=example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-2.5505084e-03 -2.9229810e-03  5.1015583e-03 ...  7.6277288e-06
   4.7286339e-03  1.7357206e-04]
 [-1.8697358e-03 -3.5421019e-03  6.3643814e-03 ... -1.2740165e-03
  -3.5635830e-04 -4.7570281e-03]
 [-6.9484329e-03 -1.4618211e-03  5.8557889e-03 ... -4.1020154e-03
   1.5374217e-03  1.4381418e-04]
 ...
 [ 6.2818634e-03  2.7013542e-03  3.0524720e-04 ... -6.8361564e-03
  -1.2974732e-02 -5.8590765e-03]
 [ 8.7941829e-03  1.0949207e-03  2.3078972e-03 ... -7.9246069e-04
  -1.3295865e-02 -4.7615073e-03]
 [ 9.0209441e-03  5.9264997e-04  9.3813613e-04 ... -1.3166096e-03
  -7.1996022e-03 -2.2286440e-03]], shape=(100, 65), dtype=float32)


In [27]:
time_pred=pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[-2.5505084e-03 -2.9229810e-03  5.1015583e-03  6.0379057e-04
 -5.5778441e-03 -1.5981396e-04  5.2391174e-03 -4.2016208e-03
  1.5229185e-03 -1.7385582e-04 -3.2914132e-03 -3.4975815e-03
 -7.6374522e-04 -1.3757800e-03 -3.1396316e-03 -8.8775071e-04
 -2.8316991e-03  2.4238245e-03 -2.5304237e-03  6.4579421e-04
 -2.5106415e-03  1.5388591e-03 -2.6674406e-03 -3.1622751e-03
 -5.8328882e-03  3.0453980e-03 -2.1885943e-03  7.1575510e-04
  4.0323823e-03 -2.8721364e-03 -2.6681863e-03 -1.3003191e-03
 -2.3369649e-03 -1.3544958e-03 -1.1968422e-03 -1.2893073e-03
 -2.2598656e-03  1.4446418e-03  4.0781945e-03 -6.0613290e-03
  3.2758941e-03  2.2682797e-03  2.6792882e-03 -1.4446389e-03
  2.1445493e-03  9.0457303e-03 -5.1061017e-03  3.8520936e-03
  1.5346400e-04  4.4075018e-03  3.5741981e-03 -4.7946861e-03
 -1.1232633e-04 -8.2447585e-03  3.0303027e-03  2.7955559e-03
  5.6577835e-04 -2.7419964e-03  4.3748929e-03 -5.1823352e-03
  4.1360655e-03  1.7851322e-03  7.6277288e-06  4.7286339e-03
  1.735720

In [28]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'olLT!ODNK3WYjLi,fvnWZm:k.atUOipkmqDlGDXGpzUHKc-w;MMiWH-.NIMBepHzib?xO!..iSsyejXWgPNEHlUuFgN to \nV\n-j'

Compiling the model

In [30]:
def loss(labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

In [31]:
model.compile(optimizer='adam',loss=loss)

Creating checkpoints

In [33]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Traning

In [34]:
history=model.fit(data,epochs=63,callbacks=[checkpoint_callback])

Epoch 1/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 16s 73ms/step - loss: 2.8576
Epoch 2/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 16s 73ms/step - loss: 1.8474
Epoch 3/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 75ms/step - loss: 1.5963
Epoch 4/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 76ms/step - loss: 1.4704
Epoch 5/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 17s 77ms/step - loss: 1.3973
Epoch 6/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - loss: 1.3439
Epoch 7/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 74ms/step - loss: 1.3059
Epoch 8/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 76ms/step - loss: 1.2693
Epoch 9/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 74ms/step - loss: 1.2343
Epoch 10/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - loss: 1.2017
Epoch 11/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - loss: 1.1695
Epoch 12/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - loss: 1.1350
Epoch 13/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 76ms/step - loss: 1.1069
Epoch 14/63
172/172 ━━━━━━━━━━━━━━━━━━━━ 16s 75ms/step - loss: 1.0671
Epoch 15/63
172/172 ━━━━━━━━━

Loading the model

In [45]:
model=build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [46]:
import tensorflow as tf, os

checkpoint_dir = './training_checkpoints'
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
print("Latest checkpoint found:", latest_checkpoint)


Latest checkpoint found: None


In [53]:
checkpoint_path = "./training_checkpoints/ckpt_10.weights.h5"
print("Attempting to load:", checkpoint_path)
model.load_weights(checkpoint_path)

Attempting to load: ./training_checkpoints/ckpt_10.weights.h5


Generating text

In [58]:
def generate_text(model,start_string):
  num_generate=800
  input_eval=[char2idx[s] for s in start_string]
  input_eval=tf.expand_dims(input_eval,0)

  text_generated=[]
  temperature=1.0

  # Access the LSTM layer and call reset_states()
  for layer in model.layers:
    if isinstance(layer, tf.keras.layers.LSTM):
      layer.reset_states()
      break

  for i in range(num_generate):
    predictions=model(input_eval)
    predictions=tf.squeeze(predictions,0)
    predictions=predictions/temperature
    prediction_id=tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()
    input_eval=tf.expand_dims([prediction_id],0)
    text_generated.append(idx2char[prediction_id])
  return(start_string+''.join(text_generated))

In [59]:
inp=input("Type a starting string: ")
print(generate_text(model,inp))

Type a starting string: Romeo
Romeong age.

GLOUCESTER:
He provided, even to many age and livit
Lady guess me and all in like death
Breaking the exite hath preterved perfuition;
For being we have relet.

Priviols
Whom your son Marguates:
Pardon wanter, rascals.

GLOUCESTER:
Soldiers art thou that be special gast, I pray thee;
For shines it best advented by some own desperaze looks! Please you, hear't; let you move merry both become.

ESCALUS:
Do you sleep? 'tis I move yet I have effeace
By offiering Blants and pawngry, yonder.

GLOUCESTER:
Then. Buck'dlasted him, you were comf;
And we refuse you.

First Servant:
Fair sisternows, my lords, my lord, we parted
That scrong'd the nose but occasion merety me.
If he never woo'd tes malking offenders.

CORIOLANUS:
No, behought your curst: how is it aw xainted:
We thank you with all
